## Check ASP Reg vs ASP fast

In [1]:
relevant_configs = {
    "miner": ["asp1-4h", "asp1-reg-4h", "asp2-4h-kb2-single", "asp2-reg-kb-4h"],
    "islands": ["asp1-4h", "asp1-reg-4h", "asp2-4h-undo", "asp2-reg-undo-4h"],
    "spiky-tireworld": ["asp1-4h","asp1-reg-4h", "asp2-4h-undo", "asp2-reg-undo-4h","asp2-reg-kb-4h"],
    "tireworld-truck": ["asp1-4h", "asp1-reg-4h", "asp2-4h-undo-single", "asp2-reg-undo-4h"],
    "doors": ["asp1-4h", "asp1-reg-4h", "asp2-4h", "asp2-reg-4h"]
    }

In [4]:
import pandas as pd
import numpy as np
LOC = "~/Work/Data/FondAsp"

In [8]:
df_sat = pd.read_csv(f"{LOC}/sat_results.csv")
df_sat = df_sat.loc[:, ~df_sat.columns.str.contains('Unnamed')]
df_sat.head()

,domain,instance,planner,type,solved,SAT,time,memory,timeout,memoryout,policysize
0,acrobatics,p06,asp1-4h,asp,False,-1,14400.383668,2158.449219,True,False,36
1,acrobatics,p06,paladinus-4h,paladinus,True,True,3.153158,82.828125,False,False,128
2,acrobatics,p06,minisat-4h,fondsat,False,-1,14400.285015,1983.265625,True,False,-1
3,acrobatics,p06,glucose-4h,fondsat,False,-1,14399.936225,610.523438,True,False,-1
4,acrobatics,p06,asp2-backbone-4h,asp,False,-1,14400.779252,2543.703125,True,False,38


### Count the total number of instances and store them

In [13]:
df_sat_counts = pd.read_csv(f"{LOC}/counts.csv", index_col="domain")
df_sat_counts

,total_count
domain,
acrobatics,8
beam-walk,11
blocksworld-ipc08,30
blocksworld-new,40
chain-of-rooms,10
doors,15
earth_observation,40
elevators,15
faults-ipc08,55


### Helper functions

In [14]:
def coverage(df_domain, df_counts):
    df_solved = df_domain.query("solved==True")
    df_solved_count = df_solved.groupby(["domain", "planner"]).count()[["instance"]].reset_index()
    df_solved_count.rename(columns={"instance": "count"}, inplace=True)
    df_coverage = df_solved_count.merge(df_counts, left_on="domain", right_on="domain", how="inner")
    df_coverage["coverage"] = df_coverage["count"]/df_coverage["total_count"]
    return df_coverage.pivot_table(index=["domain"], columns=["planner"], values=["coverage"])

def average_time_diff(df_domain, planner_1, planner_2):
    df_times = df_domain.query("solved==True").pivot_table(index=["domain", "instance"], columns=["planner"], values=["time"]).dropna()
    df_times.columns = ['_'.join(col) for col in df_times.columns.values]
    df_times["diff"] = 100*(df_times[f"time_{planner_1}"] - df_times[f"time_{planner_2}"])/df_times[f"time_{planner_1}"]
    display(df_times)
    return round(df_times[f"time_{planner_1}"].mean(),2),round(df_times[f"time_{planner_2}"].mean(),2), round(df_times['diff'].mean(),2)

def stats_kb(domain, planner_1, planner_2):
    configs = relevant_configs[domain]
    df_scenario = df_sat.query(f"domain==@domain and planner in @configs")
    df_c = coverage(df_scenario, df_sat_counts)
    p1_mean, p2_mean, percentage_diff = average_time_diff(df_scenario, planner_1, planner_2)
    
    return df_c, p1_mean, p2_mean, percentage_diff

## Scenario analysis on SAT instances

### Islands

In [16]:
df_c, p1_time, p2_time, improvement = stats_kb("islands", 'asp2-4h-undo', 'asp2-reg-undo-4h')
display(df_c)
print(p1_time, p2_time, improvement)
print(f"Average decrease in solve times by {improvement}%")

time_asp1-4h  time_asp1-reg-4h  time_asp2-4h-undo   
domain  instance                                                      
islands p01           1.163689          1.275155           0.894497  \
        p02           1.194491          1.195697           0.914682   
        p03           1.141910          1.024075           1.009045   
        p04           1.297221          1.051692           0.935371   
        p05           1.311996          1.215394           0.994023   
        p06           1.295695          1.186262           0.974685   
        p07           1.370086          1.153643           1.011788   
        p08           1.372773          1.230804           1.064807   
        p09           1.446835          1.202100           1.234331   
        p10           1.530776          1.302131           1.331419   
        p11           1.515324          1.286593           1.443615   
        p12           1.726644          1.307838           1.284235   
        p13           1.316649          1.202894           1.041715   
        p14           1.489100          1.175697           1.191661   
        p15           1.536003          1.182161           1.463993   
        p16           2.216019          1.196575           1.594560   
        p17           2.645698          1.526483           1.987794   
        p18           2.513790          1.664129           2.323541   
        p19           2.772826          1.813531           2.614001   
        p20           3.532140          2.072979           3.195511   
        p21           4.161130          2.275055           3.660543   
        p22           4.144279          2.648148           4.348851   
        p23           5.155720          3.097687           4.876903   
        p24           5.575879          3.402424           6.006287   
        p25           1.705148          1.310043           1.408972   
        p26           2.725590          1.700084           2.410671   
        p27           4.592115          2.189032           3.483159   
        p28           6.612907          2.896616           4.753311   
        p29           7.656748          3.991940           6.340905   
        p30           9.973182          5.156466           8.913251   
        p31           9.927314          6.547747          10.106126   
        p32          13.628599          8.398141          12.732590   
        p33          14.054298         10.476553          14.963408   
        p34          17.026465         13.223749          18.841679   
        p35          22.649359         17.225621          22.421102   
        p36          26.222870         21.481288          26.329580   
        p37           3.667669          2.009802           2.752581   
        p38           7.704491          3.683365           5.845204   
        p39          12.841346          6.001196          10.498446   
        p40          17.560445         10.054684          14.915637   
        p41          26.384054         15.560048          22.483925   
        p42          32.390635         24.066125          30.635692   
        p43          43.138279         34.271893          36.443625   
        p44          48.839811         47.032724          47.915830   
        p45          65.976707         67.462056          60.509215   
        p46          70.164008         91.388437          71.470831   
        p47          85.250079        118.818798          86.763586   
        p48         102.456026        142.852284         100.131356   
        p49           8.278214          4.396681           6.293829   
        p50          21.523726         13.442741          14.714991   
        p51          37.119735         27.971523          28.367230   
        p52          51.710278         55.167012          46.144874   
        p53          73.406928         98.491641          64.333076   
        p54         100.735834        141.488986          90.597704   
        p55         128.706453        204.

coverage                                          
planner  asp1-4h asp1-reg-4h asp2-4h-undo asp2-reg-undo-4h
domain                                                    
islands      1.0         1.0          1.0              1.0

37.69 23.33 25.55
Average decrease in solve times by 25.55%


### Miner

In [262]:
df_c, p1_time, p2_time, improvement = stats_kb("miner", 'asp1-4h', 'asp1-4h-kb')
display(df_c)
print(p1_time, p2_time, improvement)
print(f"Average decrease in solve times by {improvement}%")

time_asp1-4h  time_asp1-4h-kb  time_asp1-4h-undo       diff
domain instance                                                             
miner  p01          18.707134        17.892042          17.284011   4.357119
       p02          16.076967        15.135138          14.574727   5.858253
       p03          20.045592        19.333486          19.568227   3.552429
       p04          74.401413        79.408012          78.042042  -6.729172
       p05          69.450128        73.652528          69.467090  -6.050960
       p07         157.143603       161.137660         162.451868  -2.541661
       p08         118.538299       120.444250         127.198438  -1.607877
       p09         157.851991       168.399388         171.036193  -6.681828
       p10         110.130568       120.344347         126.592267  -9.274246
       p11         124.578368       121.832011         126.436822   2.204522
       p12          74.528633        74.514239          74.822833   0.019313
       p13          85.167154        90.518674          85.109152  -6.283549
       p14         169.395533       207.999971         205.705968 -22.789526
       p15         206.021501       219.965519         228.906555  -6.768235
       p16         209.792574       248.858218         244.157353 -18.621081
       p17         239.061168       281.804068         302.228880 -17.879483
       p18         183.488007       198.115265         218.238561  -7.971779
       p19         425.613644       501.847758         548.400071 -17.911577
       p20         286.250793       314.372850         343.119816  -9.824272
       p21         396.612827       432.441897         444.192076  -9.033765
       p22         651.669466       652.570804         681.115875  -0.138312
       p23         413.702961       480.140382         484.909949 -16.059208
       p24         597.700828       680.497114         682.064190 -13.852463
       p25         252.617577       280.587653         279.173330 -11.072102
       p26         255.806790       292.308566         301.681196 -14.269275
       p27         486.642464       516.333685         545.955184  -6.101239
       p28         432.562960       472.585662         489.540168  -9.252457
       p29         311.423296       334.872964         339.473163  -7.529837
       p30         214.990530       249.981798         253.948800 -16.275725
       p31         236.404090       258.505048         272.293343  -9.348805
       p32        1070.601287      1049.961388         997.217404   1.927879
       p33         655.832682       731.728057         713.800341 -11.572369
       p34         393.940576       378.906166         477.492995   3.816416
       p35         420.369528       418.565670         532.035360   0.429112
       p36         375.740863       335.636988         447.867767  10.673280
       p37         490.588972       424.421155         570.151040  13.487424
       p38         977.765567       745.654043         996.632428  23.738975
       p39        1170.402049       913.761282        1111.193871  21.927573
       p40         765.122059       615.374338         769.436869  19.571743
       p41         696.243409       576.315599         739.832118  17.224983
       p42         613.307739       514.516238         693.031448  16.107982
       p43         819.248599       668.474200         665.017792  18.403986
       p44         681.324398       587.870381         576.004411  13.716523
       p46        1549.705391      1109.485809        1124.254201  28.406663
       p50        1326.003717       818.045814        1108.990192  38.307427

coverage                        
planner  asp1-4h asp1-4h-kb asp1-4h-undo
domain                                  
miner       0.96       0.94          0.9

422.28 390.56 -0.48
Average decrease in solve times by -0.48%


### Spiky Tireworld


In [263]:
df_c, p1_time, p2_time, improvement = stats_kb("spiky-tireworld", 'asp1-4h', 'asp1-4h-undo')
display(df_c)
print(p1_time, p2_time, improvement )
print(f"Average decrease in solve times by {improvement}%")

time_asp1-4h  time_asp1-4h-undo       diff
domain          instance                                            
spiky-tireworld p01         222.185609          73.735810  66.813418
                p02         394.077962         114.040968  71.061318
                p03         546.400103         151.657801  72.244185
                p04          60.646958          25.660017  57.689523
                p05         776.226829         251.125232  67.647958
                p06        1127.264979         310.496727  72.455746
                p07        1258.458311         384.827293  69.420736
                p08        1613.661388         446.857107  72.307876
                p09        1834.835714         556.143481  69.689740
                p10        2096.901949         598.881418  71.439703

coverage             
planner           asp1-4h asp1-4h-undo
domain                                
spiky-tireworld  0.909091          1.0

993.07 291.34 69.08
Average decrease in solve times by 69.08%


### Tireworld Truck

In [264]:
df_c, p1_time, p2_time, improvement = stats_kb("tireworld-truck", 'asp1-4h', 'asp1-4h-undo')
display(df_c)
print(p1_time, p2_time, improvement)
print(f"Average decrease in solve times by {improvement}%")

time_asp1-4h  time_asp1-4h-undo       diff
domain          instance                                            
tireworld-truck p01           1.640760           1.764090  -7.516679
                p02           1.454048           1.528061  -5.090102
                p03           1.896543           1.657823  12.587101
                p04           2.543349           2.373627   6.673163
                p05           3.630763           3.413070   5.995806
...                                ...                ...        ...
                p71         210.946242         125.632983  40.443128
                p72         965.528965         522.881821  45.845040
                p73         215.594749         167.694391  22.217776
                p74          81.644215          71.099423  12.915541
                p75          39.910197          36.534682   8.457777

[74 rows x 3 columns]

coverage             
planner          asp1-4h asp1-4h-undo
domain                               
tireworld-truck      1.0          1.0

47.98 33.36 17.26
Average decrease in solve times by 17.26%
